In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import random
import numpy as np
import matplotlib.pylab as plt
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import os
import math
import openbabel
import pybel
import itertools
from scipy.stats import expon
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, rdMolDescriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.ChemicalFeatures import BuildFeatureFactory
from collections import OrderedDict, Counter
from scipy.stats import randint as sp_randint
from sklearn import preprocessing
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, ShuffleSplit, StratifiedKFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from ggplot import *
from time import time
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 40, 40

def reading_csv():
    csv_file = pd.read_csv("lista.csv", index_col=None, decimal=",", na_values="ND")
    return csv_file


def data_preproc(df):
    df_preproc = df.ix[:, ['ID', 'SMILES', 'ION_ACTIVITY', 'TEST_SET', 'PvsP']]
    # Dropping every row with a NaN value in any of the columns (both PvsP and IRON_ACTIVITY)
    df_preproc = df_preproc.dropna(subset=['ID', 'SMILES', 'ION_ACTIVITY'])
    df_preproc.loc[:, 'ION_ACTIVITY'] = pd.to_numeric(df_preproc.loc[:, 'ION_ACTIVITY'], errors='coerce')
    # Transforming PvsP (Potency vs Parasite (uMol) to log10(x)
    df_preproc['pEC50'] = np.log10(df_preproc['PvsP'])

    print ("Number of instances of each level of ION_ACTIVITY", "\n", df_preproc['ION_ACTIVITY'].value_counts())

    return df_preproc


def series4_cmp(df):
    id_series4 = pd.read_csv("osm_series4.csv", index_col=None)
    osm_series4 = pd.merge(df, id_series4, how='inner', on=['SMILES'])

    print("Number of instances of each level of ION_ACTIVITY", "\n", osm_series4['ION_ACTIVITY'].value_counts())

    return osm_series4


def dragon_cmp(df):
    norm_dragon = pd.read_csv("kellerberrin/NormDragon.csv", index_col=None)
    norm_dragon = pd.merge(df, norm_dragon, how='inner', on='SMILES')
    return norm_dragon


def smiles_to_rdkit(smiles):
    mols = []
    mols = [Chem.MolFromSmiles(x) for x in smiles]
    return mols

def smiles_to_pybel(smiles):
    mols = []
    mols = [pybel.readstring("smi",x) for x in smiles]
    return mols

# Getting RDKit descriptors
def rdkit_desc(df):
    descs = [x[0] for x in Chem.Descriptors.descList]
    two = ['SMILES', 'ION_ACTIVITY', 'TEST_SET']
    col_r = two + descs
    rddf = pd.DataFrame(columns=col_r, index=df.index.values)
    rddf = rddf.fillna(0)
    rddf.loc[:, ("SMILES", "ION_ACTIVITY", "TEST_SET")] = df.loc[:, ("SMILES", "ION_ACTIVITY", "TEST_SET")]
    mrdk_list = smiles_to_rdkit(rddf.loc[:, "SMILES"])
    rddf = rddf.assign(MOLECULES=mrdk_list)
    list_descriptors = []
    mdc = MolecularDescriptorCalculator(descs)
    for index, row in rddf.iterrows():
        results = mdc.CalcDescriptors(row["MOLECULES"])
        descriptors = dict(zip(descs, results))
        descriptors.update(
            {'SMILES': row["SMILES"], 'ION_ACTIVITY': row["ION_ACTIVITY"], 'MOLECULES': row["MOLECULES"], 'TEST_SET': row["TEST_SET"]})
        list_descriptors.append(descriptors)

    rdkit_df = pd.DataFrame(list_descriptors)
    return rdkit_df

#Getting Pybel descriptors
def pybel_desc(df):
    descs = pybel.descs
    two = ['SMILES', 'ION_ACTIVITY']
    col_r = two + descs
    pydf = pd.DataFrame(columns=col_r, index=df.index.values)
    pydf = pydf.fillna(0)
    pydf.loc[:, ("SMILES", "ION_ACTIVITY")] = df.loc[:, ("SMILES", "ION_ACTIVITY")]
    mpy_list = smiles_to_pybel(pydf.loc[:, "SMILES"])
    pydf = pydf.assign(MOLECULES=mpy_list)
    list_descriptors = []
    for index, row in pydf.iterrows():
        results = row["MOLECULES"].calcdesc()
        results.update(
            {'SMILES': row["SMILES"], 'ION_ACTIVITY': row["ION_ACTIVITY"], 'MOLECULES': row["MOLECULES"]})
        list_descriptors.append(results)

    pybel_df = pd.DataFrame(list_descriptors)
    pybel_df = pybel_df.dropna(axis=1, how = 'all')
    pybel_df = pybel_df.drop(['MOLECULES'], axis=1)
    return pybel_df
    

def sanitize_smiles(df):
    for index, row in df.iterrows():
        try:
            result = Chem.SanitizeMol(row["MOLECULES"])
            if result != Chem.SanitizeFlags.SANITIZE_NONE:
                sanitized_smile = Chem.MolToSmiles(row["MOLECULES"])
                print("Sanitized SMILE %s, Compound ID:%s", sanitized_smile)
                df.set_value(index, "SMILES", sanitized_smile)
        except:
            print("Unable to Sanitize SMILE %s, Compound ID:%s", row["SMILES"])
            print("Record Deleted.")
            df.drop(index, inplace=True)
    return df


# Fingerprint descriptor calculator. Rdkit default values
#def fingerprint_desc(df):
#    fp_descs = ['MORGAN1024', 'MORGAN2048_1', 'MORGAN2048_2', 'MORGAN2048_3', 'MORGAN2048_4', 'MORGAN2048_5',
#                'MORGAN2048_6', 'TOPOLOGICAL2048', 'MACCFP']
#    two = ['SMILES', 'ION_ACTIVITY']
#    col_f = two + fp_descs
#    fpd = pd.DataFrame(columns=col_f, index=df.index.values)
#    fpd = fpd.fillna(0)
#    fpd.loc[:, ("SMILES", "ION_ACTIVITY")] = df.loc[:, ("SMILES", "ION_ACTIVITY")]
#    mrdk_list = smiles_to_rdkit(fpd.loc[:, "SMILES"])
#    fpd = fpd.assign(MOLECULES=mrdk_list)
#    list_fpdescriptors = []
#    for index, row in fpd.iterrows():
#        morgan_1024 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 4, nBits=1024)
#        morgan_2048_1 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 1, nBits=2048)
#        morgan_2048_2 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 2, nBits=2048)
#        morgan_2048_3 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 3, nBits=2048)
#        morgan_2048_4 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 4, nBits=2048)
#        morgan_2048_5 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 5, nBits=2048)
#        morgan_2048_6 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 6, nBits=2048)
#        topological_2048 = AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect(row["MOLECULES"])
#        macc = AllChem.GetMACCSKeysFingerprint(row["MOLECULES"])
#        fp_descriptors = {'SMILES': row["SMILES"], 'ION_ACTIVITY': row["ION_ACTIVITY"], 'MOLECULES': row["MOLECULES"],
#                          'MORGAN1024': morgan_1024, 'MORGAN2048_1': morgan_2048_1, 'MORGAN2048_2': morgan_2048_2,
#                          'MORGAN2048_3': morgan_2048_3, 'MORGAN2048_4': morgan_2048_4, 'MORGAN2048_5': morgan_2048_5,
#                          'MORGAN2048_6': morgan_2048_6, 'TOPOLOGICAL2048': topological_2048, 'MACCFP': macc}
#        list_fpdescriptors.append(fp_descriptors)
#    fp_df = pd.DataFrame(list_fpdescriptors)
#    return fp_df


def merge_dfs(df1, df2):
    df_merge = pd.merge(df1, df2, how='inner', on=('SMILES', 'ION_ACTIVITY'))
    # Checking duplicated rows
    df_merge = df_merge.drop_duplicates(subset='SMILES')

    return df_merge


def df_to_ndarrays(df):
    df.select_dtypes(include=['object'])

    # Managing data types
    #if(df['RDF125v'].dtype == 'object'):
    #    df['RDF125v'] = df['RDF125v'].str.replace(',', '.')
    #    df['RDF125v'] = pd.to_numeric(df['RDF125v'])
    #if(df['RDF100p'].dtype == 'object'):
    #    df['RDF100p'] = pd.to_numeric(df['RDF100p'])
    #if (df['RDF140p'].dtype == 'object'):
    #    df['RDF140p'] = pd.to_numeric(df['RDF140p'])
    # Removing compounds with activity = 0.5 (not statistically valuable)
    df = df.drop(df[df.ION_ACTIVITY == 0.5].index)
    #df = df.drop(['SMILES', 'ID', 'TEST_SET', 'pEC50', 'PvsP', 'MOLECULES'], axis=1)
    #df = df.drop(['SMILES', 'TEST_SET', 'MOLECULES'], axis=1)
    
    df = df.drop(['SMILES', 'MOLECULES'], axis=1)
    #df = df.drop('SMILES', axis=1)
    
    # Removing non-numeric variables
    npdesc = df.drop(['ION_ACTIVITY'], axis=1).values
    labdesc = df.ix[:, "ION_ACTIVITY"].values
    # Scaling data
    #npdesc = preprocessing.scale(npdesc)
    return npdesc, labdesc


def splitting_as_competition(df, fp=False):
    df.select_dtypes(include=['object'])
        
    df = df.drop(df[df.ION_ACTIVITY == 0.5].index)
        
    osm_training = df.ix[(df['TEST_SET'] != 'B') & (df['TEST_SET'] != 'C'), :]
    osm_testing = df.ix[(df['TEST_SET'] == 'B') | (df['TEST_SET'] == 'C'), :]
        
    osm_training = osm_training.drop(['SMILES', 'TEST_SET', 'MOLECULES'], axis=1)
    osm_testing = osm_testing.drop(['SMILES', 'TEST_SET', 'MOLECULES'], axis=1)
    
    if fp == True:
        osm_training_fp = osm_training.ix[:,fp_names]
        osm_testing_fp = osm_testing.ix[:,fp_names]
        
        osm_training = osm_training.drop(fp_names, axis=1)
        osm_testing = osm_testing.drop(fp_names, axis=1)
        
        nptrain_fp = osm_training_fp.as_matrix()
        nptest_fp = osm_testing_fp.as_matrix()
    
    nptrain = osm_training.drop(['ION_ACTIVITY'], axis=1).as_matrix()
    labtrain = osm_training.ix[:, "ION_ACTIVITY"].as_matrix()
    nptest = osm_testing.drop(['ION_ACTIVITY'], axis=1).as_matrix()
    labtest = osm_testing.ix[:, "ION_ACTIVITY"].as_matrix()
    
    # Scaling data
    nptrain = preprocessing.scale(nptrain)
    nptest = preprocessing.scale(nptest)
    
    if fp == True:
        nptrain = np.concatenate((nptrain, nptrain_fp), axis=1)
        nptest = np.concatenate((nptest, nptest_fp), axis=1)
    
    return nptrain, nptest, labtrain, labtest
    

# def splitting_as_competition(df):
#     df.select_dtypes(include=['object'])

#     #if(df['RDF125v'].dtype == 'object'):
#     #    df['RDF125v'] = df['RDF125v'].str.replace(',', '.')
#     #    df['RDF125v'] = pd.to_numeric(df['RDF125v'])
#     #if(df['RDF100p'].dtype == 'object'):
#     #    df['RDF100p'] = pd.to_numeric(df['RDF100p'])
#     #if (df['RDF140p'].dtype == 'object'):
#     #    df['RDF140p'] = pd.to_numeric(df['RDF140p'])
        
#     df = df.drop(df[df.ION_ACTIVITY == 0.5].index)

#     osm_training = df.ix[(df['TEST_SET'] != 'B') & (df['TEST_SET'] != 'C'), :]
#     osm_testing = df.ix[(df['TEST_SET'] == 'B') | (df['TEST_SET'] == 'C'), :]


#     osm_training = osm_training.drop(['SMILES', 'TEST_SET', 'MOLECULES'], axis=1)
#     osm_testing = osm_testing.drop(['SMILES', 'TEST_SET', 'MOLECULES'], axis=1)
    
#     #########################################################################
#     #included 06/06
#     osm_training_fp = osm_training.ix[:,fp_names]
#     osm_testing_fp = osm_testing.ix[:,fp_names]
    
#     osm_training = osm_training.drop(fp_names, axis=1)
#     osm_testing = osm_testing.drop(fp_names, axis=1)
    
#     #########################################################################
#     nptrain = osm_training.drop(['ION_ACTIVITY'], axis=1).as_matrix()
#     labtrain = osm_training.ix[:, "ION_ACTIVITY"].as_matrix()
#     nptest = osm_testing.drop(['ION_ACTIVITY'], axis=1).as_matrix()
#     labtest = osm_testing.ix[:, "ION_ACTIVITY"].as_matrix()
    
#     #########################################################################
#     #included 06/06
#     nptrain_fp = osm_training_fp.as_matrix()
#     nptest_fp = osm_testing_fp.as_matrix()
#     #########################################################################
    
#     # Scaling data
#     nptrain = preprocessing.scale(nptrain)
#     nptest = preprocessing.scale(nptest)
    
#     #########################################################################
#     #included 06/06
#     nptrain = np.concatenate((nptrain, nptrain_fp), axis=1)
#     nptest = np.concatenate((nptest, nptest_fp), axis=1)
#     #########################################################################
#     return nptrain, nptest, labtrain, labtest

def balancing_classes(npdesc, labdesc, method):
    print("Original dataset shape {}".format(Counter(labdesc)))
    # Goal: balancing the dataset
    if method == "smote":
        mod = SMOTE(random_state=30)
    elif method == "rov":
        mod = RandomOverSampler(random_state=30)
    elif method == "adasyn":
        mod = ADASYN(random_state=30)
    else:
        print("The oversampling method introduced is not available")

    npdesc_res, labdesc_res = mod.fit_sample(npdesc, labdesc)
    print('Resampled dataset shape {}'.format(Counter(labdesc_res)))
    return npdesc_res, labdesc_res


def feature_sel(npdesc_original, npdesc_bal, labdesc_original, esti):
    if esti == "svm_lin":
        estimator = SVC(kernel="linear")
    elif esti == "log":
        estimator = LogisticRegression()
    else:
        print("The estimation method introduced is not available")
    # Filtering half of the features
    selector = RFECV(estimator, cv=5,)
    # Using original data to perform feature selection
    npdesc_filt = selector.fit(npdesc_original, labdesc_original)
    print("Optimal number of features: %d" % npdesc_filt.n_features_)

    # Filtering selected features from the data array
    npdesc_fs = npdesc_bal[:, npdesc_filt.support_]
    return npdesc_fs

def splitting_data(npdesc,labdesc, test_size):
    x_train, x_test, y_train, y_test = train_test_split(npdesc, labdesc, test_size=test_size, random_state=0)
    return x_train, x_test, y_train, y_test

def classification(x_train, x_test, y_train, y_test, modelo):
    # splitting data in 80% training set, 20% test set
    # x_train, x_test, y_train, y_test = train_test_split(npdesc, labdesc, test_size=0.3, random_state=0)
    # Logistic regression
    if modelo == "log_res":
        params = {"C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
        #model = LogisticRegressionCV(penalty='l2', cv=10)
        ran_search = GridSearchCV(estimator=LogisticRegression(penalty='l2',intercept_scaling=1, dual=False, 
                                                               fit_intercept=True, tol=0.0001), 
                                  param_grid=params, cv=10)
        ran_search = ran_search.fit(x_train, y_train)
        model = ran_search.best_estimator_
        print("Best parameters set found on development set:", ran_search.best_params_)

    # SVC
    elif modelo == "svm":
        # params = [{'C': stats.expon(scale=10), 'gamma': stats.expon(scale=0.1), 'kernel': ['linear']},
        #          {'C': stats.expon(scale=10), 'gamma': stats.expon(scale=0.1), 'kernel': ['rbf']},
        #          {'C': stats.expon(scale=10), 'gamma': stats.expon(scale=0.1), 'kernel': ['sigmoid']},
        #          {'C': stats.expon(scale=10), 'gamma': stats.expon(scale=0.1), 'degree': [2, 3], 'kernel': ['poly']}]
        # params = {'C': expon(scale=10), 'gamma': expon(scale=0.1), 'kernel': [str('linear'), str('rbf'), str('sigmoid'), str('poly')], 'degree':[2, 3]}
        # Randomized search
        params = {'C': [1e-3, 1e-2, 1e-1, 1, 5, 10, 100], 'gamma': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
                  'kernel': [str('linear'), str('rbf'), str('sigmoid'), str('poly')], 'degree': [2, 3]}
        #ran_search = RandomizedSearchCV(estimator=SVC(probability=True), param_distributions=params, cv=10, n_iter=100, n_jobs=1)
        ran_search = GridSearchCV(estimator=SVC(probability=True), param_grid=params, cv=10)
        start = time()
        ran_search = ran_search.fit(x_train, y_train)
        print("RandomizedSearchCV took %.2f seconds" % (time() - start))
        model = ran_search.best_estimator_
        print("Best parameters set found on development set:", ran_search.best_params_)
        print("Grid scores on development set:")
        means = ran_search.cv_results_['mean_test_score']
        stds = ran_search.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, ran_search.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    # Random Forests
    elif modelo == "rforest":
        # Designate distributions to sample hyperparameters from
        #n_estimators = np.random.uniform(70, 80, 5).astype(int)
        #max_features = np.random.normal(6, 3, 5).astype(int)
        # Check max_features>0 & max_features<=total number of features
        #max_features[max_features <= 0] = 1
        #max_features[max_features > x_train.shape[1]] = x_train.shape[1]
        # hyperparameters = {'n_estimators': list(n_estimators),
        #                   'max_features': list(max_features)}
        hyperparameters = {'n_estimators': [10, 50, 100, 150, 200, 250, 300], 'max_depth': sp_randint(1,x_train.shape[1]),
                           'max_features': sp_randint(1,x_train.shape[1]), 'criterion': [str('gini'), str('entropy')],
                           'bootstrap':[True, False], "min_samples_split": sp_randint(2, x_train.shape[1]),
                           'min_samples_leaf': [1, 10, 20, 30, 40, 50]}
        ran_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=hyperparameters, n_iter=100, n_jobs=1, cv=10)
        ran_search = ran_search.fit(x_train, y_train)
        model = ran_search.best_estimator_
        print("Best parameters set found on development set:", ran_search.best_params_)

    # Nearest Neighbours
    elif modelo == "knn":
        params = {"n_neighbors": [1, 3, 5, 7, 9], "metric": [str("euclidean"), str("manhattan"), str("chebyshev"), str("minkowski")], "p":[2]}
        ran_search = GridSearchCV(KNeighborsClassifier(algorithm='auto', p=2), param_grid=params)
        ran_search = ran_search.fit(x_train, y_train)
        model = ran_search.best_estimator_
        print("Best parameters set found on development set:", ran_search.best_params_)

    else:
        print("The prediction model introduced is not available")

    model = model.fit(x_train, y_train)

    probs = model.predict_proba(x_test)

    # if modelo == "log_res":
    #     print(pd.DataFrame(zip(x_test.columns, np.transpose(modelo.coef_))))
    
    print("MODEL USED: ", modelo)

    # Accuracy on the training set
    print('Accuracy on the training set:', model.score(x_train, y_train))

    y_pred = model.predict(x_test)
    
    print ("The accuracy predicting in the test set is ", metrics.accuracy_score(y_test, y_pred))
    print ("The AUC of the ROC curve is", metrics.roc_auc_score(y_test, probs[:, 1]))
    print("Confusion matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))
    print("Detailed classification report:")
    print (metrics.classification_report(y_test, y_pred))

    # Computing ROC curve
    fpr, tpr, _ = metrics.roc_curve(y_test, probs[:, 1])

    df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
    x = ggplot(df, aes(x='fpr', y='tpr')) + \
        geom_line() + \
        geom_abline(linetype='dashed') + \
        scale_x_continuous(limits=(0,1)) + \
        scale_y_continuous(limits=(0,1)) + \
        xlab('FPR') + \
        ylab('TPR') + \
        ggtitle("ROC Curve w/ AUC=%s" % str(metrics.auc(fpr,tpr)))
    return x
    
    
def xgboost_prueba(alg, x_train, x_test, y_train, y_test, cv_folds=5, early_stopping_rounds=50):
    # loading numpy array into dMatrix
    xgb_train = xgb.DMatrix( x_train, label=y_train)
    xgb_test = xgb.DMatrix( x_test, label=y_test)
    
    #Train cross validation
    xgb_param = alg.get_xgb_params()
    
    cvresult = xgb.cv(xgb_param, xgb_train, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                      metrics = ['logloss'], early_stopping_rounds=early_stopping_rounds)
    alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(x_train, y_train, eval_metric='auc')
    #alg.fit(x_train, y_train, eval_metric='logloss')
    
    #Predict in training set
    y_trpred = alg.predict(x_train)
    y_trpredprob = alg.predict_proba(x_train)[:,1]
    
    #Predict in test set
    y_pred = alg.predict(x_test)
    y_predprob = alg.predict_proba(x_test)[:,1]
    
    #Print model report:
    print ("\nModel Report")
    
    print ("Accuracy (train set): %.4g" % metrics.accuracy_score(y_train, y_trpred))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, y_trpredprob))
    
    print ("Accuracy (test set): %.4g" % metrics.accuracy_score(y_test, y_pred))
    print ("AUC Score (Test): %f" % metrics.roc_auc_score(y_test, y_predprob))    
    
    #parte añadida que puede petar
    ################################################################################
    print("Confusion matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))
    print("Detailed classification report:")
    print (metrics.classification_report(y_test, y_pred))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    
        # Computing ROC curve
    fpr, tpr, _ = metrics.roc_curve(y_test, y_predprob)

    df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
    x = ggplot(df, aes(x='fpr', y='tpr')) + \
        geom_line() + \
        geom_abline(linetype='dashed') + \
        scale_x_continuous(limits=(0,1)) + \
        scale_y_continuous(limits=(0,1)) + \
        xlab('FPR') + \
        ylab('TPR') + \
        ggtitle("ROC Curve w/ AUC=%s" % str(metrics.auc(fpr,tpr)))
    return x
    

    
def xgboost_tuning(x_train, y_train, rat):
    #cr_va = 3
    random.seed(6)
    cr_va = StratifiedKFold(n_splits=3, shuffle=False, random_state=1)
    
    #Add feature selection part
    
    #Define "baseline" model
    #xgbl = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1,gamma=0, subsample=0.8, 
    #                     colsample_bytree=0.8, objective= 'binary:logistic', scale_pos_weight=1, seed=27)
    
    #Tuning max_depth and min_child_weight
    param_test1 = {'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2)}

    gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5, 
                                                      min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, 
                                                      #objective= 'binary:logistic', scale_pos_weight=1, seed=27),
                                                      objective= 'binary:logistic', scale_pos_weight=rat),
                           # param_grid = param_test1, scoring='roc_auc',iid=False, cv=5)
                            param_grid = param_test1, scoring='roc_auc',iid=False, cv=cr_va)
    
    gsearch1.fit(x_train,y_train)
    print(gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_)
    max_depth = gsearch1.best_params_['max_depth']
    min_child_weight= gsearch1.best_params_['min_child_weight'] 
    
    #Tuning gamma
    param_test2 = {'gamma':[i/10.0 for i in range(0,5)]}
    gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=max_depth, 
                                                      min_child_weight= min_child_weight, gamma=0, subsample=0.8, 
                                                      colsample_bytree=0.8, objective= 'binary:logistic', 
                                                      #scale_pos_weight=1,seed=27), 
                                                      scale_pos_weight=rat), 
                           # param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
                            param_grid = param_test2, scoring='roc_auc',iid=False, cv=cr_va)
                            
    gsearch2.fit(x_train,y_train)
    print(gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_)
    gamma= gsearch2.best_params_['gamma']
    
    #Tuning subsample and colsample_bytree
    param_test3 = {'subsample':[i/10.0 for i in range(6,10)], 'colsample_bytree':[i/10.0 for i in range(6,10)]}
    gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=max_depth,
                                                      min_child_weight=min_child_weight, gamma=gamma, subsample=0.8, colsample_bytree=0.8,
                                                      #objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
                                                      objective= 'binary:logistic', scale_pos_weight=rat), 
                            # param_grid = param_test3, scoring='roc_auc', iid=False, cv=5)
                            param_grid = param_test3, scoring='roc_auc', iid=False, cv=cr_va)
    gsearch3.fit(x_train, y_train)
    print(gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_)
    subsample = gsearch3.best_params_['subsample']
    colsample_bytree = gsearch3.best_params_['colsample_bytree']
    
    #Retuning subsample and colsample_bytree
    param_test3b = {'subsample':[i/100.0 for i in range(int(subsample*100)-5,95,5)], 
                    'colsample_bytree':[i/100.0 for i in range(int(colsample_bytree*100)-5,95,5)]}
    gsearch3b = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=max_depth,
                                                       min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, 
                                                       colsample_bytree=colsample_bytree,
                                                      #objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
                                                       objective= 'binary:logistic', scale_pos_weight=rat),
                            # param_grid = param_test3, scoring='roc_auc', iid=False, cv=5)
                            param_grid = param_test3b, scoring='roc_auc', iid=False, cv=cr_va)
    gsearch3b.fit(x_train, y_train)
    print(gsearch3b.grid_scores_, gsearch3b.best_params_, gsearch3b.best_score_)
    subsample = gsearch3b.best_params_['subsample']
    colsample_bytree = gsearch3b.best_params_['colsample_bytree']
    
    #Tuning reg_alpha 
    param_test4 = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}

    gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=max_depth,
                                                      min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, 
                                                      colsample_bytree=colsample_bytree, objective= 'binary:logistic',  
                                                      #scale_pos_weight=1,seed=27), 
                                                      scale_pos_weight=rat), 
                        # param_grid = param_test4, scoring='roc_auc', iid=False, cv=5)
                            param_grid = param_test4, scoring='roc_auc', iid=False, cv=cr_va)
    gsearch4.fit(x_train,y_train)
    print(gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_)
    reg_alpha = gsearch4.best_params_['reg_alpha']
    
    #Tuning learning_rate and n_estimators
    param_test5= {'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3], 'n_estimators' : [100, 200, 300, 400, 500]}
    gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=max_depth,
                                                      min_child_weight=min_child_weight, gamma=gamma, subsample=subsample, 
                                                      colsample_bytree=colsample_bytree, objective= 'binary:logistic',  
                                                      #scale_pos_weight=1,seed=27),
                                                     scale_pos_weight=rat),
                        # param_grid = param_test5, scoring='roc_auc', iid=False, cv=5)
                        param_grid = param_test5, scoring='roc_auc', iid=False, cv=cr_va)

    gsearch5.fit(x_train,y_train)
    print(gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_)
    learning_rate = gsearch5.best_params_['learning_rate']
    n_estimators = gsearch5.best_params_['n_estimators']
    
    optim_model = XGBClassifier(learning_rate = learning_rate ,n_estimators= n_estimators, max_depth=max_depth,
                                min_child_weight=min_child_weight,gamma=gamma,subsample=subsample,
                                colsample_bytree=colsample_bytree, reg_alpha=reg_alpha, objective= 'binary:logistic',
                                #scale_pos_weight=1,seed=27)
                                scale_pos_weight=rat)
    
    return optim_model


In [5]:
#########################################################################################################################
#Fingerprint calculator
nbits = 1024
longbits = 16384

# dictionary
fpdict = {}
#fpdict['ecfp0'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 0, nBits=nbits)
#fpdict['ecfp2'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 1, nBits=nbits)
#fpdict['ecfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=nbits)
fpdict['ecfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=nbits)
#fpdict['ecfc0'] = lambda m: AllChem.GetMorganFingerprint(m, 0)
#fpdict['ecfc2'] = lambda m: AllChem.GetMorganFingerprint(m, 1)
#fpdict['ecfc4'] = lambda m: AllChem.GetMorganFingerprint(m, 2)
#fpdict['ecfc6'] = lambda m: AllChem.GetMorganFingerprint(m, 3)
#fpdict['fcfp2'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 1, useFeatures=True, nBits=nbits)
#fpdict['fcfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, useFeatures=True, nBits=nbits)
fpdict['fcfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, useFeatures=True, nBits=nbits)
#fpdict['fcfc2'] = lambda m: AllChem.GetMorganFingerprint(m, 1, useFeatures=True)
#fpdict['fcfc4'] = lambda m: AllChem.GetMorganFingerprint(m, 2, useFeatures=True)
#fpdict['fcfc6'] = lambda m: AllChem.GetMorganFingerprint(m, 3, useFeatures=True)
#fpdict['lecfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=longbits)
#fpdict['lecfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=longbits)
#fpdict['lfcfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, useFeatures=True, nBits=longbits)
#fpdict['lfcfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, useFeatures=True, nBits=longbits)

#fpdict['maccs'] = lambda m: MACCSkeys.GenMACCSKeys(m)

#fpdict['ap'] = lambda m: Pairs.GetAtomPairFingerprint(m)
#fpdict['tt'] = lambda m: Torsions.GetTopologicalTorsionFingerprintAsIntVect(m)
fpdict['hashap'] = lambda m: rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(m, nBits=nbits)
fpdict['hashtt'] = lambda m: rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(m, nBits=nbits)
#fpdict['avalon'] = lambda m: fpAvalon.GetAvalonFP(m, nbits)
#fpdict['laval'] = lambda m: fpAvalon.GetAvalonFP(m, longbits)
#fpdict['rdk5'] = lambda m: Chem.RDKFingerprint(m, maxPath=5, fpSize=nbits, nBitsPerHash=2)
#fpdict['rdk6'] = lambda m: Chem.RDKFingerprint(m, maxPath=6, fpSize=nbits, nBitsPerHash=2)
fpdict['rdk7'] = lambda m: Chem.RDKFingerprint(m, maxPath=7, fpSize=nbits, nBitsPerHash=2)

fp_names = fpdict.keys()

# Fingerprint descriptor calculator. Rdkit default values
def fingerprint_desc(df):
    #fp_descs = ['MORGAN1024', 'MORGAN2048_1', 'MORGAN2048_2', 'MORGAN2048_3', 'MORGAN2048_4', 'MORGAN2048_5',
    #            'MORGAN2048_6', 'TOPOLOGICAL2048', 'MACCFP']
    two = ['SMILES', 'ION_ACTIVITY']
    col_f = two + fp_names
    fpd = pd.DataFrame(columns=col_f, index=df.index.values)
    fpd = fpd.fillna(0)
    fpd.loc[:, ("SMILES", "ION_ACTIVITY")] = df.loc[:, ("SMILES", "ION_ACTIVITY")]
    mrdk_list = smiles_to_rdkit(fpd.loc[:, "SMILES"])
    fpd = fpd.assign(MOLECULES=mrdk_list)
    list_fpdescriptors = []
    for index, row in fpd.iterrows():
        results = {}
        for fp in fp_names:
            results[fp] = fpdict[fp](row["MOLECULES"])
        np_fps = {}
        for key, value in results.iteritems():
            arr = np.zeros((1,))
            if isinstance(value, DataStructs.cDataStructs.ExplicitBitVect):
                DataStructs.ConvertToNumpyArray(value, arr)
            else:
                if isinstance(value, DataStructs.cDataStructs.UIntSparseIntVect):
                    fp_bin = DataStructs.cDataStructs.UIntSparseIntVect.ToBinary(value)
                elif isinstance(value, DataStructs.cDataStructs.IntSparseIntVect):
                    fp_bin = DataStructs.cDataStructs.IntSparseIntVect.ToBinary(value)
                else:
                    fp_bin = DataStructs.cDataStructs.LongSparseIntVect.ToBinary(value)
                fp_frombin = DataStructs.cDataStructs.CreateFromBinaryText(fp_bin)
                DataStructs.ConvertToNumpyArray(fp_frombin, arr)
            np_fps[key] = arr.tolist()
        #morgan_1024 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 4, nBits=1024)
        #morgan_2048_1 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 1, nBits=2048)
        #morgan_2048_2 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 2, nBits=2048)
        #morgan_2048_3 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 3, nBits=2048)
        #morgan_2048_4 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 4, nBits=2048)
        #morgan_2048_5 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 5, nBits=2048)
        #morgan_2048_6 = AllChem.GetMorganFingerprintAsBitVect(row["MOLECULES"], 6, nBits=2048)
        #topological_2048 = AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect(row["MOLECULES"])
        #macc = AllChem.GetMACCSKeysFingerprint(row["MOLECULES"])
            np_fps.update({'SMILES': row["SMILES"], 'ION_ACTIVITY': row["ION_ACTIVITY"], 'MOLECULES': row["MOLECULES"]})
        #fp_descriptors = {'SMILES': row["SMILES"], 'ION_ACTIVITY': row["ION_ACTIVITY"], 'MOLECULES': row["MOLECULES"],
        #                  'MORGAN1024': morgan_1024, 'MORGAN2048_1': morgan_2048_1, 'MORGAN2048_2': morgan_2048_2,
        #                  'MORGAN2048_3': morgan_2048_3, 'MORGAN2048_4': morgan_2048_4, 'MORGAN2048_5': morgan_2048_5,
        #                  'MORGAN2048_6': morgan_2048_6, 'TOPOLOGICAL2048': topological_2048, 'MACCFP': macc}
        list_fpdescriptors.append(np_fps)
    fp_df = pd.DataFrame(list_fpdescriptors)
    
    ################################################################
    #inserted 06/06
    for index, row in fp_df.iterrows():
        for fp in fp_names:
            ar_val = np.array(fp_df.ix[index, fp])
            fp_df.set_value(index, fp, ar_val)
            
    fp_df = fp_df.drop(['MOLECULES'], axis=1)
    
    return fp_df

In [55]:
def spectrophores(df):
    """
    Spectrophores descriptor calculator.
    """
    two = ['SMILES', 'ION_ACTIVITY']
    sp_names = [j+str(i) for i,j in zip(range(1,49),["sp"] * 48)]
    #col_s = two + ['SPECTROPHORES'] + sp_names
    col_s = two + sp_names
    spd = pd.DataFrame(columns=col_s, index=df.index.values)
    spd = spd.fillna(0)
    spd.loc[:, ("SMILES", "ION_ACTIVITY")] = df.loc[:, ("SMILES", "ION_ACTIVITY")]
    mpy_list = smiles_to_pybel(spd.loc[:, "SMILES"])
    spd = spd.assign(MOLECULES=mpy_list)
    #spd['SPECTROPHORES'] = spd['SPECTROPHORES'].astype(object)
    #list_spdescriptors = []
    
    for index, row in spd.iterrows():
        row["MOLECULES"].make3D()
        spectrophore = pybel.ob.OBSpectrophore()
        spectrophore.SetNormalization(spectrophore.NormalizationTowardsZeroMeanAndUnitStd)
        hey = spectrophore.GetSpectrophore(row["MOLECULES"].OBMol)
        sp_descriptors = {n: d for n, d in zip(sp_names, hey)}
        
        for key in sp_descriptors.keys():
            spd.loc[index, key] = sp_descriptors[key]
    
    
    spd = spd.dropna(axis=1, how='all')
    return spd

In [ ]:
def fp_individual(df):  
    fp_names_short = ['rdk7', 'rdk5', 'hashtt', 'ecfp6', 'ecfp4', 'ecfp2', 'ecfp0', 'fcfp2', 'fcfp4', 'rdk6', 'fcfp6', 'hashap']
    fp_names_long = ['lfcfp6', 'lfcfp4', 'lecfp4', 'lecfp6']
    fp_names_ss =  ['maccs']
    
    for index, row in df.iterrows():
        for name in fp_names:
            if name in fp_names_short:
                name_fp = [j+"_"+str(i) for i, j in zip(range(1,nbits+1), [name]*(nbits+1))]
            
            elif name in fp_names_long:
                name_fp = [j+"_"+str(i) for i, j in zip(range(1,longbits+1), [name]*(longbits+1))]
            
            else:
                name_fp = [j+"_"+str(i) for i, j in zip(range(1,167+1), [name]*(167+1))]
            hey = df.loc[index, name]
            fps = {n: d for n, d in zip(name_fp, hey)}
            for key in fps.keys():
                df.loc[index, key] = fps[key]
    return df

In [1]:
def quitar_fps(df):
    fp_df = df.drop(fp_names, axis=1)
    return fp_df